## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.uniform(shape=[28*28, 100], minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(tf.zeros(shape=[100]))
        self.W2 = tf.Variable(tf.random.uniform(shape=[100, 10], minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(tf.zeros(shape=[10]))
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = tf.reshape(x, (-1, 28*28))
        h1 = tf.nn.tanh(x @ self.W1 + self.b1)
        logits = h1 @ self.W2 + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [6]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.04986 ; accuracy 0.46481666
epoch 1 : loss 2.045468 ; accuracy 0.4712
epoch 2 : loss 2.0410898 ; accuracy 0.47731668
epoch 3 : loss 2.0367253 ; accuracy 0.48283333
epoch 4 : loss 2.0323744 ; accuracy 0.48798335
epoch 5 : loss 2.028036 ; accuracy 0.49345
epoch 6 : loss 2.0237107 ; accuracy 0.49838334
epoch 7 : loss 2.019398 ; accuracy 0.5037
epoch 8 : loss 2.015097 ; accuracy 0.5084
epoch 9 : loss 2.0108082 ; accuracy 0.5133833
epoch 10 : loss 2.006531 ; accuracy 0.5174
epoch 11 : loss 2.002265 ; accuracy 0.52178335
epoch 12 : loss 1.9980104 ; accuracy 0.52613336
epoch 13 : loss 1.9937668 ; accuracy 0.53078336
epoch 14 : loss 1.9895339 ; accuracy 0.5349
epoch 15 : loss 1.9853115 ; accuracy 0.53945
epoch 16 : loss 1.9810992 ; accuracy 0.5432
epoch 17 : loss 1.9768974 ; accuracy 0.54715
epoch 18 : loss 1.9727054 ; accuracy 0.5510833
epoch 19 : loss 1.9685231 ; accuracy 0.5549667
epoch 20 : loss 1.9643503 ; accuracy 0.55873334
epoch 21 : loss 1.9601871 ; accuracy 0.56185
e